In [8]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import matplotlib.pyplot as plt
from matplotlib import animation
from sim_functions_ver4 import *
import json
with open("scenarios.txt") as f: scenarios = json.load(f)
    
def inputDate(x):
    if (x >= dt.time(0,0,0)) & (x < dt.time(6,0,0)):
        return dt.datetime.combine(dt.date(2019, 8, 7), x)
    else:
        return dt.datetime.combine(dt.date(2019, 8, 6), x)
    
def handleDF(sim):
    df = sim.copy(deep=True)
    df['time'] = sim['time'].apply(inputDate)
    df = df.set_index(['time','car'])
    return df.T.stack().T

def plotFigures(name, sim, algo):
    testDF = handleDF(sim)
    
    fig4, ax4 = plt.subplots(figsize=(20,8))
    
    for i in range(4):
        ax4.scatter(testDF.index, np.ones(len(testDF.index))*i, 
                    s=(testDF['batt'][i]**2.1).apply(int))

    for j in range(len(testDF.index)):
        ax4.axvline(x=testDF.index[j], ymin=-1, ymax=1, 
                    color='black', linestyle='-', lw=0.5, alpha=0.2)

    ax4.yaxis.set_ticks(np.arange(0,4,1))
    ax4.set_title('Batt (kW)', {'fontsize': 20})
    ax4.set_xlim([dt.datetime(2019,8,6,5,0,0), dt.datetime(2019,8,7,6,0,0)])
    plt.savefig('figures/' + name + '_' + algo + '_rate.png')
    
def createAnimation(name, sim, variable, algo):
    df = handleDF(sim)
    
    fig3, ax3 = plt.subplots(figsize=(13,6))
    ax3.set_xlim(dt.datetime(2019,8,6,5,0,0), dt.datetime(2019,8,7,6,0,0))
    ax3.grid(color='black', linestyle='-', linewidth=0.4)
    if variable == 'batt':
        ax3.set_ylim(0, 31)
    else:
        ax3.set_ylim(-5, 8)
        ax3.axhline(y=7.5, xmin=-1, xmax=1, color='black', linestyle='--', lw=2.5)
        ax3.axhline(y=-0.5, xmin=-1, xmax=1, color='black', linestyle='--', lw=2.5)
        
    xdata, ydata = [], [[], [], [], []]
    ln1,ln2,ln3,ln4 = plt.plot([], [], '.-b', [], [], '.-y', [], [], '.-g', [], [], '.-r')

    def init(): return ln1,ln2,ln3,ln4

    def update(frame):
        xdata.append(frame)
        for i in range(len(ydata)): ydata[i].append(df[variable][i][frame])
        ln1.set_data(xdata, ydata[0])
        ln2.set_data(xdata, ydata[1])
        ln3.set_data(xdata, ydata[2])
        ln4.set_data(xdata, ydata[3])
        return ln1,ln2,ln3,ln4

    ani = animation.FuncAnimation(fig3, update, frames=df.index.values, init_func=init, 
                                  interval=1000, repeat=False, blit=True)
    ani.save('animations/' + name + '_' + algo + '_' + variable + '.mp4', fps=1, extra_args=['-vcodec', 'libx264'])

In [9]:
chunks = getChunks()
startTime = readTime("06:00")   # CHOOSE STARTTIME
runTime = 24                    # CHOOSE RUNTIME (HRS)
mpkw = 4                        # SET AVERAGE MILES PER kW THAT WILL DETERMINE RATE OF BATT DECREASE
mph = 16                        # SET AVERAGE MILES PER HR COVERED
car_cols = ["battPerc","inDepot","battSize","chargePt"]
cp_cols = ["maxRate","inUse"]
sim_cols = ['time','car','charge_rate','batt','event']
performance = pd.DataFrame(columns=['type','dumb','leavetime','batt','smart'])

for filename in scenarios:
    carShifts = scenarios[filename]['carShifts']
    chargeCapacity = scenarios[filename]['chargeCapacity']
    carData = scenarios[filename]['carData']
    chargePtData = scenarios[filename]['chargePtData']

    dumbDF, dumb_sim, dumb_rc = runSimulation(
                            startTime, runTime,
                            carData, car_cols, carShifts,
                            chargePtData, cp_cols, chargeCapacity, 
                            sim_cols, mph, mpkw, 'dumbCharge')

    smart_leavetimeDF, smart_leavetime_sim, smart_leavetime_rc = runSimulation(
                            startTime, runTime,
                            carData, car_cols, carShifts,
                            chargePtData, cp_cols, chargeCapacity, 
                            sim_cols, mph, mpkw, 'smartCharge_leavetime')

    smart_battDF, smart_batt_sim, smart_batt_rc = runSimulation(
                            startTime, runTime,
                            carData, car_cols, carShifts,
                            chargePtData, cp_cols, chargeCapacity, 
                            sim_cols, mph, mpkw, 'smartCharge_batt')

    smartDF, smart_sim, smart_rc = runSimulation(
                            startTime, runTime,
                            carData, car_cols, carShifts,
                            chargePtData, cp_cols, chargeCapacity, 
                            sim_cols, mph, mpkw, 'superSmartCharge')
    
#     performance = performance.append({
#         'type': filename,
#         'dumb': dumb_rc,
#         'leavetime': smart_leavetime_rc,
#         'batt': smart_batt_rc,
#         'smart': smart_rc
#     }, ignore_index=True)

#     ###############################################################
#     # SAVE TO EXCEL (ONLY RUN WHEN ALL ALGORITHMS ARE UNCOMMENTED)
#     # NOTE: CREATE A FOLDER CALLED 'RESULTS' FIRST
#     ###############################################################
#     # open writer
#     writer = pd.ExcelWriter("results/" + filename + ".xlsx")
#     # write files
#     dumbDF.to_excel(
#         writer, sheet_name="dumb")
#     smart_leavetimeDF.to_excel(
#         writer, sheet_name="smart_leavetime")
#     smart_battDF.to_excel(
#         writer, sheet_name="smart_batt")
#     smartDF.to_excel(
#         writer, sheet_name="superSmart")
#     # close writer
#     writer.save()
        
#     ###############################################################
#     # SAVE TO PNG (ONLY RUN WHEN ALL ALGORITHMS ARE UNCOMMENTED)
#     # NOTE: CREATE A FOLDER CALLED 'FIGURES' FIRST
#     ###############################################################
#     plotFigures(filename, dumb_sim, 'dumb')
#     plotFigures(filename, smart_leavetime_sim, 'leavetime')
#     plotFigures(filename, smart_batt_sim, 'batt')
#     plotFigures(filename, smart_sim, 'smart')
#     plt.close('all')

#     ###############################################################
#     # SAVE TO MP4 (ONLY RUN WHEN ALL ALGORITHMS ARE UNCOMMENTED)
#     # NOTE: CREATE A FOLDER CALLED 'ANIMATIONS' FIRST
#     ###############################################################
#     createAnimation(filename, dumb_sim, 'batt', 'dumb')
#     createAnimation(filename, smart_leavetime_sim, 'batt', 'leavetime')
#     createAnimation(filename, smart_batt_sim, 'batt', 'batt')
#     createAnimation(filename, smart_sim, 'batt', 'smart')
#     plt.close('all')
    
#     createAnimation(filename, dumb_sim, 'charge_rate', 'dumb')
#     createAnimation(filename, smart_leavetime_sim, 'charge_rate', 'leavetime')
#     createAnimation(filename, smart_batt_sim, 'charge_rate', 'batt')
#     createAnimation(filename, smart_sim, 'charge_rate', 'smart')
#     plt.close('all')
    
    print(str(filename) + ' done')
    
# writer = pd.ExcelWriter("results/comparison.xlsx")
# performance.to_excel(writer)
# writer.save()

# smart_sim[['event','batt']].to_excel('output.xlsx')

04_cars_goods_delivery done
04_cars_meal_delivery done
04_cars_taxi done
04_cars_national_express done
04_cars_first_shift_same done
04_cars_second_shift_same done
